create the results table for the accessibility paper

In [ ]:
import geopandas
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [ ]:
perimeters = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    index='id'
)

regions = perimeters[
    perimeters.index.isin([
        'zurich', 'ebc_zrh_v01_ex_zurich', 'zurich_seefeld'
    ])
]

regions

In [ ]:
accessibility = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility_diff.gpkg')
)

accessibility = gpd.sjoin(
    accessibility,
    regions,
    how='left',
    predicate='within'
).rename(columns={'index_right': 'region'})

accessibility

Add group labels

In [ ]:
accessibility['x'] = 'all'
accessibility['y'] = 'all'

accessibility['age_group'] = accessibility.apply(
    lambda row: 'age<=25' if row['age'] <= 25 else 'age>=60' if row['age'] >= 60 else 'age:other',
    axis=1
)

accessibility['nationality'] = accessibility.apply(
    lambda row: 'nationality:swiss' if row['residencepermit'] <= -2 else 'nationality:other',
    axis=1
)

accessibility['maritalstatus_type'] = accessibility.apply(
    lambda row: 'maritalstatus:married' if row['maritalstatus'] == 2 else 'maritalstatus:other',
    axis=1
)

accessibility['sex_type'] = accessibility.apply(
    lambda row: 'sex:male' if row['sex'] == 1 else 'sex:female' if row['sex'] == 2 else 'sex:other',
    axis=1
)

stats_values = ['median']

addition = ''

fields = {
    'x': ['count'],
    f'accessibility_{MODE_PRIVATE_CARS}_before{addition}': stats_values,
    f'accessibility_{MODE_TRANSIT}_before{addition}': stats_values,
    f'accessibility_{MODE_CYCLING}_before{addition}': stats_values,
    f'accessibility_{MODE_FOOT}_before{addition}': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}_after{addition}': stats_values,
    f'accessibility_{MODE_TRANSIT}_after{addition}': stats_values,
    f'accessibility_{MODE_CYCLING}_after{addition}': stats_values,
    f'accessibility_{MODE_FOOT}_after{addition}': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}{addition}_diff_perc': stats_values,
    f'accessibility_{MODE_TRANSIT}{addition}_diff_perc': stats_values,
    f'accessibility_{MODE_CYCLING}{addition}_diff_perc': stats_values,
    f'accessibility_{MODE_FOOT}{addition}_diff_perc': stats_values,
    f'accessibility_before{addition}': stats_values,
    f'accessibility_after{addition}': stats_values,
    f'accessibility{addition}_diff_perc': stats_values,
}

accessibility_stats = {
    'region_and_age': accessibility.groupby(['region', 'age_group']).agg(fields),
    'region_and_nationality': accessibility.groupby(['region', 'nationality']).agg(fields),
    'region_and_sex': accessibility.groupby(['region', 'sex_type']).agg(fields),
    'all': accessibility.groupby(['x', 'y']).agg(fields)
}

res = pd.concat(accessibility_stats.values())
res = res.iloc[::-1]

#for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
#    res[f'accessibility_{mode}_diff'] = (
#        res[f'accessibility_{mode}_diff'] / res[f'accessibility_{mode}_before']
#    )
    
#res[f'accessibility_diff'] = (
#        res[f'accessibility_diff'] / res[f'accessibility_before']
#    )

res = res.sort_index().transpose()

res.to_excel(
    os.path.join(outputs_path, 'accessibility_results_transposed.xlsx'),
)

res

In [ ]:
import math

math.log10(0.1)